In [ ]:
import pandas as pd
import sqlite3


def agregar_cantidad_horas_programadas(df:pd.DataFrame,año:int,mes:int,dias_feriados_del_periodo:int=0, cant_horas_diarias:int = 8)-> pd.DataFrame:
    """Se agrega una nueva columna con la cantidad de horas en días de semana"""
    import calendar
    from datetime import date

    def obtener_dias_laborales(anio, mes):
        # Obtener el número de días en el mes
        num_dias = calendar.monthrange(anio, mes)[1]

        # Listar todos los días del mes
        dias_laborales = []
        for dia in range(1, num_dias + 1):
            fecha = date(anio, mes, dia)
            # Si no es sábado (5) ni domingo (6), es laboral
            if fecha.weekday() < 5:
                dias_laborales.append(fecha)

        return dias_laborales

    # obtener las fechas de los días laborales en el periodo.
    dias_laborales = obtener_dias_laborales(año, mes)
    # calculo de las horas en dias laborales del periodo
    df['horas_programadas'] = (len(dias_laborales)-dias_feriados_del_periodo)*cant_horas_diarias
    return df

def generate_programacion_horas_perc_excel(
                            ruta_formato_horas,
                            año,
                            mes,
                            filepath="/",
                            dias_feriado= 0,
                            cant_horas_diarias = 8
):
    # importar formato de empleados para plantilla
    df = pd.read_excel(ruta_formato_horas)

    # Sección de extraer el dni_perc, tener en cuenta que la base que tengo actualmente tiene - en algunos.
    # limpiar antes de implementar la función, también restringir en el ingreso de datos del software.
    df['dni_perc'] = df['Empleado'].str.split('__').str[1]

    # agregar la cantidad de horas programadas (Dias hábiles por cantidad de horas al día)
    df_base_perc = agregar_cantidad_horas_programadas(df,año,mes,dias_feriado, cant_horas_diarias)
    df_base_perc = df_base_perc[['Empleado','Total Empleados', 'Total Pagado', 'Componente Salarial','dni_perc','horas_programadas']].copy()


    # Traer la distribución por unidades de producción desde la base de datos
    conn = sqlite3.connect('rrhh.db')

    query = """
    SELECT 
        -- Concatenación de nombre y código
        up.codigo_up || '-' || up.nombre_up AS unidad_produccion,
        c.dni_perc,
        COALESCE(dc.porcentaje,0) AS porcentaje
    FROM cat_unidades_produccion AS up
    LEFT JOIN distribucion_costos AS dc ON up.id_unidad = dc.id_unidad
    LEFT JOIN contratos AS c ON dc.id_contrato = c.id_contrato;
    """
    dist_up = pd.read_sql_query(query, conn)

    conn.close()


    # Uniendo para mantener todas las unidades, necesarias para la dinamización.
    base = pd.merge(
            df_base_perc,
            dist_up,
            on='dni_perc',
            how='right'
            )
    # -------------------------------------------------------------------
    # Cantidad de horas --- está pendiente quitar permisos (Inasistencias)
    base['horas'] = base['horas_programadas']*(base['porcentaje']/100)
    # -------------------------------------------------------------------
    # Solo columnas necesarias
    base = base[['Empleado', 'Total Empleados', 'Total Pagado', 'Componente Salarial','unidad_produccion','horas']].fillna({'horas':0})
    # dinamizando las columnas para tener todos las unidades de producción como en la plantilla de la plataforma PERC
    df_pivot = base.pivot(
        index=['Empleado', 'Total Empleados', 'Total Pagado', 'Componente Salarial'],
        columns='unidad_produccion',
        values='horas'
    ).reset_index()
    # Exportación en estructura de la plantilla para la plataforma
     # 4. Exportar a Excel
  
    with pd.ExcelWriter(f'Programación_Horas_{año}_{mes}.xlsx', engine='openpyxl') as writer:
        df_pivot[df_pivot['Empleado'].notna()].to_excel(writer, index=False, sheet_name='Programación Hora')
        
        # Auto-ajustar ancho de columnas (Opcional, estética)
        worksheet = writer.sheets['Programación Hora']
        for column_cells in worksheet.columns:
            length = max(len(str(cell.value)) for cell in column_cells)
            worksheet.column_dimensions[column_cells[0].column_letter].width = length + 2
    

In [ ]:
generate_programacion_horas_perc_excel(
ruta_formato_horas = r"C:\Users\mmarrder\Downloads\formato_horas_perc_2025_12.xlsx",
año = 2025,
mes = 12,
dias_feriado = 3,
cant_horas_diarias = 8
)

# agregando las unidades correctas a la base de datos

In [ ]:
# import pandas as pd
# import sqlite3

# # 1. Supongamos que tu dataframe se llama 'df'
# # df = pd.DataFrame(...) 

# # 2. Establecer la conexión a la base de datos SQLite
# # Sustituye 'tu_base_de_datos.db' por la ruta de tu archivo
# conn = sqlite3.connect('rrhh.db')

# try:
#     # 3. Exportar el dataframe a la tabla de SQLite
#     unidades[['codigo_up','nombre_up']].to_sql(
#         name='cat_unidades_produccion', # Nombre de la tabla en SQLite
#         con=conn,                       # La conexión que abrimos
#         if_exists='append',             # 'append' agrega los datos, 'replace' borra la tabla y la crea de nuevo
#         index=False                     # No guarda el índice de pandas como una columna extra
#     )
#     print("Datos insertados correctamente.")

# except Exception as e:
#     print(f"Ocurrió un error: {e}")

# finally:
#     # 4. Cerrar la conexión
#     conn.close()

In [ ]:
# unidades_lista_completa = ['120-Hospitalizacion nefrologia pediatrica',
#  '170-Unidad de cuidados intensivos pediatria',
#  '183-Unidad de cuidados intermedios pediatria',
#  '733-Consulta gastroenterologia pediatrica',
#  '734-Consulta nutricion pediatrica',
#  '319-Consulta otorrinolaringologia',
#  '331-Consulta neurologia pediatrica',
#  '332-Consulta nefrologia pediatrica',
#  '336-Consulta dermatologia pediatrica',
#  '340-Consulta endocrinologia pediatrica',
#  '739-Consulta psicologia pediatrica',
#  '1726-Consulta Inmunología y Alergías Pediátrica',
#  '1725-Consulta Cardiología y Hemodinámia Pediátrica',
#  '1727-Consulta Cirugía General Pediátrica',
#  '1728-Consulta Anestesiología Pediatrica',
#  '463-Quirofanos cardiologia',
#  '923-Quirofanos hemodinamia',
#  '1731-Quirófano Miscelaneos',
#  '630-Apoyo a otros establecimientos',
#  '1730-Hospitalización Miscelaneos Pediátrica',
#  '2088-Sala de Lactantes',
#  '1743-Unidad de estabilización',
#  '535-Radiologia',
#  '586-Dialisis peritoneal',
#  '268-Hemodialisis',
#  '593-Servicio farmaceutico',
#  '629-Cuidados paliativos',
#  '644-Ambulancia',
#  '662-Central de esterilizacion',
#  '712-Mantenimiento biomedico',
#  '648-Aseo',
#  '664-Transporte general',
#  '665-Mantenimiento',
#  '1742-Trabajo social / Unidad de atención al usuario',
#  '670-Administracion']

In [ ]:
# import pandas as pd
# unidades = pd.DataFrame({'unidades':unidades_lista_completa})

In [ ]:
# unidades['unidades'].str.split('-').str[0]

In [ ]:
# unidades['codigo_up'] = unidades['unidades'].str.split('-').str[0]
# unidades['nombre_up'] = unidades['unidades'].str.split('-').str[1]

In [ ]:
# unidades[['codigo_up','nombre_up']].to_sql('rrhh.db',)